In [ ]:
import pandas as pd
import numpy as np
store_file = 'allsommarioni_owners standardised forms_dataset_V2.xlsx'
# in the same file there are corrections & new additions to the transcription
SPLIT_IDX = 154
new_transc = pd.read_excel(store_file, sheet_name='LETTERE ')

In [ ]:
# two lines in the middle that isabella put in blank to make a separation needs to be removed before integrating the new transcription
new_transc = pd.concat([new_transc.iloc[:SPLIT_IDX], new_transc.iloc[SPLIT_IDX+2:]]).rename(columns={'area': 'page'})

replace_d = {'numero_della_mappa': 'parcel_number',
             'subalterno': 'sub_parcel_number',
              'corr_as': 'austrian_cadaster_correspondance',
              'corr_ai': 'austro_italian_cadaster_correspondance',
              'denom_pezzi_di_terra':'house_number',
              'possessore': 'owner',
              'possessore_standardised': 'owner_standardised',
              'qualità': 'quality',
              'uniqueID': 'unique_id',
              'standardised_fonction': 'qualities',
              'place_acronym': 'district_acronym'
}
img_to_double_letter_pn = {
    "reg6bis/0177": ["RA", "RB", "RC", "RD", "RE", "RF", "RG", "RH", "RI", "RJ", "RK", "RL"],
    "reg6bis/0178": ["RM", "RN", "RO", "RP", "RQ", "RR", "RS", "RT", "RU", "RV", "RW", "RX", "RY", "RZ", "SA", "SB", "SC", "SD", "SE"],
    "reg6bis/0179": ["SF", "SG", "SH", "SI", "SJ", "SK", "SL", "SM"],
    "reg6bis/0180": ["SN", "SO","SP", "SQ", "SR", "SS", "ST", "SU"],
    "reg6bis/0181": ["SV", "SW", "SX", "SY", "SZ", "TA", "TB", "TC", "TD", "TE", "TF"],
    "reg6bis/0182": ["TG", "TH", "TI", "TJ", "TK", "TL", "TM", "TN", "TO", "TP", "TQ", "TR", "TS", "TT", "TU", "TV", "TW", "TX", "TY", "TZ", "UA"],
    "reg6bis/0183": ["UB", "UC", "UD", "UE", "UF", "UG", "UH", "UI", "UJ", "UK", "UL", "UM"],
    "reg6bis/0184": ["UN", "UO", "UP", "UQ", "UR", "US", "UT", "UU", "UV", "UW", "UX", "UY", "UZ", "VA", "VB", "VC"],
    "reg6bis/0185": ["VD", "VE", "VF", "VG", "VH", "VI", "VJ", "VK", "VL", "VM", "VN", "VO", "VP", "VQ", "VR"],
    "reg6bis/0186": ["VS", "VT", "VU", "VV", "VW", "VX", "VY", "VZ", "XA", "XB", "XC", "XD"]
}

def parcel_number_to_page(pn: str) -> str:
    for img, pn_list in img_to_double_letter_pn.items():
        if pn in pn_list:
            return img
    return None

new_transc = new_transc.rename(columns=replace_d)

new_transc['parcel_number'] = new_transc['parcel_number'].str.strip()
# the new transcription leaves parcel_number empty in sub-parcel number following the sub-parcel-number of 1.0.
new_transc['parcel_number'] = new_transc['parcel_number'].ffill()
# to make them match to the geometries
new_transc['parcel_number'] = new_transc['parcel_number'].replace('UJ', 'UI').replace('SJ', 'SI')

new_transc['sub_parcel_number'] = new_transc['sub_parcel_number'].apply(lambda v: str(int(v)) if not np.isnan(v) else v)
new_transc['ownership_types'] = [[]] * len(new_transc)
new_transc['page'] = new_transc['parcel_number'].apply(parcel_number_to_page)
new_transc['qualities'] = new_transc['qualities'].apply(lambda l: [v.strip().upper() for v in l.split(',')] if type(l) is str else l)

In [ ]:
import pandas as pd 
import geopandas as gpd
from pathlib import Path

sommarioni_p = Path('../../1808_Sommarioni/')
sommarioni_geo_fp = list(sommarioni_p.rglob('sommarioni_geometries_202*.geojson'))[0]
sommarioni_txt_fp = list(sommarioni_p.rglob('sommarioni_text_data_with_pages_202*.json'))[0]
df = pd.read_json(sommarioni_txt_fp)
gdf = gpd.read_file(sommarioni_geo_fp)

In [ ]:
parcel_number_to_geom_id = gdf.set_index('parcel_number')['geometry_id'].to_dict()
# interesting mistake between the redaction of the map and the registry, a LG on the map was written where it should have been "LC" as written correctly in the registry
# this is the only case where the geometry_id is not a bijection with the parcel numbers. 
lg_id = gdf[gdf['parcel_number'] == 'LG'].geometry_id.values[0]
parcel_number_to_geom_id.update({'LC': lg_id})

new_transc['geometry_id'] = new_transc['parcel_number'].apply(lambda v: parcel_number_to_geom_id[v] if v in parcel_number_to_geom_id else None)
new_transc['geometry_id'].isna().sum()

In [ ]:
# need to find districts for each of those new transcriptions, using the centroid of the geometry of the corresponding parcel and the contemporary maps to find them.
sgdf = gpd.read_file('../../contemporary_maps/2024_Sestiere_EPSG32633.geojson').to_crs('EPSG:4326')

def sestiere_from_single_gps(gps, sgdf):
    for i, row in sgdf.iterrows():
        if row['geometry'].contains(gps):
            return row['division_name']
    return None

new_transc_geoms = gdf[gdf['geometry_id'].isin(new_transc['geometry_id'].dropna())].dissolve(by='geometry_id')
new_transc_geoms['coordinate'] = new_transc_geoms.centroid
new_transc_geoms['sestiere'] = new_transc_geoms.apply(lambda row: sestiere_from_single_gps(row['coordinate'], sgdf), axis=1)
new_transc_geoms['area'] = new_transc_geoms.area

division_name_to_acronym = {
    "SAN POLO": "NSP",
    "CANNAREGIO":"NCN",
	"SANTA CROCE":"NSC",
	"DORSODURO":"NDD",
	"SAN MARCO":"NSM",
	"CASTELLO":"NCS",
}
new_transc_geoms['district_acronym'] = new_transc_geoms['sestiere'].apply(lambda v: division_name_to_acronym[v] if v in division_name_to_acronym else None)
geom_id_to_district_acronym = new_transc_geoms.reset_index().set_index('geometry_id')['district_acronym'].to_dict()
new_transc['district_acronym'] = new_transc['geometry_id'].apply(lambda v: geom_id_to_district_acronym[v] if v in geom_id_to_district_acronym else None)

In [ ]:
# need to compute the area of each of the building of the new transcription
new_transc_geoms = new_transc_geoms.to_crs('EPSG:32633')
new_transc_geoms['area'] = new_transc_geoms.area
geom_id_to_area = new_transc_geoms.reset_index().set_index('geometry_id')['area'].to_dict()

new_transc['area'] = new_transc['geometry_id'].apply(lambda v: geom_id_to_area[v] if v in geom_id_to_area else None)

In [ ]:
# adding the unique id to the new transcriptions, to make the uuid already generated the same, we just add the new transcription to the range
max_unique_id = int(df.unique_id.max())
new_transc['unique_id'] = range(max_unique_id, max_unique_id + len(new_transc))
df['unique_id'] = df['unique_id'].astype(int)
df['new_transcription'] = False
new_transc['new_transcription'] = True
df = pd.concat([df, new_transc], ignore_index=True)


In [ ]:
df.to_json('3_Sommarioni_with_new_transcriptions_20250226.json', orient='records', lines=True)